In [6]:
#Connect to the drive for the database
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#red heart
import numpy as np, trimesh

scene = trimesh.load('/content/drive/MyDrive/Major_Project/outputs/heart_gt_patient_1001_ct.obj', force='scene')

red = np.array([255, 0, 0, 255], dtype=np.uint8)

for name, geom in scene.geometry.items():
    if isinstance(geom, trimesh.Trimesh):
        # ensure we use color visuals (not textures) so colors apply
        if hasattr(geom, 'visual') and hasattr(geom.visual, 'to_color'):
            geom.visual = geom.visual.to_color()
        geom.visual.vertex_colors = np.tile(red, (len(geom.vertices), 1))

from IPython.display import display, HTML
from trimesh.viewer import scene_to_notebook

html = scene_to_notebook(scene, height=500)
display(html)


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#testing visualisation
import trimesh
scene = trimesh.load('/content/drive/MyDrive/Major_Project/outputs/heart_gt_patient_1001_ct.obj', force='scene')
scene.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#api key for local tunnel
from pyngrok import ngrok
# Set your authtoken (replace with your actual token)
ngrok.set_auth_token("34179TiGrIN8E3BvcYO03oYiYrD_2sMD4DGizK9Xek4yk3G9x")
# Then create tunnel
public_url = ngrok.connect(PORT)

NameError: name 'PORT' is not defined

In [13]:
#before running install modules
!pip install open3d
!pip install trimesh
!pip install flask_cors
!pip install fast-simplification
!pip install pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 108.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 78.8 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.2 MB/s eta 0:00:00


In [14]:
#run before my file Password
!curl -s https://loca.lt/mytunnelpassword

34.74.89.56

In [15]:
# ==============================================================================
# 3D HEART RECONSTRUCTION SYSTEM - COMPLETE CODE FOR GOOGLE COLAB
# ==============================================================================
# Run this in Google Colab with GPU enabled (Runtime -> Change runtime type -> T4 GPU)
# ==============================================================================

import os
import io
import numpy as np
import nibabel as nib
from scipy.ndimage import zoom
from skimage import measure
import trimesh
from flask import Flask, request, jsonify, send_file
from flask_cors import CORS
import threading
from IPython.display import display, HTML
import time
import subprocess

# ==============================================================================
# CONFIGURATION
# ==============================================================================
BASE_DIR = "/content/drive/MyDrive/Major_Project"
DATA_DIR = f"{BASE_DIR}/MM-WHS 2017 Dataset"
OUTPUT_DIR = f"{BASE_DIR}/outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

AVAILABLE_PATIENTS = list(range(1001, 1011))  # 1001 to 1010
PORT = 5000

# ==============================================================================
# HELPER FUNCTIONS
# ==============================================================================
def load_nifti(path):
    """Load NIfTI file and return as numpy array"""
    img = nib.load(path)
    return img.get_fdata().astype(np.float32)

def resample_to_shape(volume, target_shape=(128, 128, 128), is_label=True):
    """Resample volume to target shape"""
    factors = [t / s for s, t in zip(volume.shape, target_shape)]
    order = 0 if is_label else 1
    return zoom(volume, factors, order=order)

def generate_ground_truth_mesh(patient_id, modality='ct'):
    """Generate ground truth 3D mesh from medical image"""
    try:
        # Construct file paths
        if modality == 'ct':
            label_path = f"{DATA_DIR}/ct_train/ct_train_{patient_id}_label.nii.gz"
        else:
            label_path = f"{DATA_DIR}/mr_train/mr_train_{patient_id}_label.nii.gz"

        # Check if file exists
        if not os.path.exists(label_path):
            return None, f"Label file not found for patient {patient_id}"

        # Load and process label
        print(f"Loading label from: {label_path}")
        label = load_nifti(label_path)

        # Resample to standard size
        label_resampled = resample_to_shape(label, target_shape=(128, 128, 128), is_label=True)

        # Convert to binary mask
        binary_mask = (label_resampled > 0).astype(np.uint8)

        # Generate mesh using marching cubes
        print("Generating 3D mesh...")
        verts, faces, normals, values = measure.marching_cubes(binary_mask, level=0.5)

        # Create trimesh object
        mesh = trimesh.Trimesh(vertices=verts, faces=faces, vertex_normals=normals)

        # Save to file
        output_filename = f"heart_gt_patient_{patient_id}_{modality}.obj"
        output_path = os.path.join(OUTPUT_DIR, output_filename)
        mesh.export(output_path)

        print(f"Mesh saved to: {output_path}")

        # Generate preview data (simplified mesh for web viewing)
        mesh_simplified = mesh.simplify_quadric_decimation(face_count=5000)

        return {
            'success': True,
            'patient_id': patient_id,
            'modality': modality,
            'filename': output_filename,
            'filepath': output_path,
            'vertices_count': len(verts),
            'faces_count': len(faces),
            'mesh_data': {
                'vertices': mesh_simplified.vertices.tolist()[:1000],
                'faces': mesh_simplified.faces.tolist()[:1000]
            }
        }, None

    except Exception as e:
        print(f"Error generating mesh: {str(e)}")
        import traceback
        traceback.print_exc()
        return None, str(e)

# ==============================================================================
# FLASK APPLICATION
# ==============================================================================
app = Flask(__name__)
CORS(app)

@app.route('/')
def index():
    """Serve the main HTML page"""
    return '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>3D Heart Reconstruction System</title>
    <style>
        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }

        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
            padding: 20px;
        }

        .container {
            max-width: 1400px;
            margin: 0 auto;
            background: white;
            border-radius: 20px;
            box-shadow: 0 20px 60px rgba(0,0,0,0.3);
            overflow: hidden;
        }

        .header {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 40px;
            text-align: center;
        }

        .header h1 {
            font-size: 2.5em;
            margin-bottom: 10px;
            text-shadow: 2px 2px 4px rgba(0,0,0,0.2);
        }

        .header p {
            font-size: 1.2em;
            opacity: 0.9;
        }

        .main-content {
            display: grid;
            grid-template-columns: 350px 1fr;
            gap: 0;
            min-height: 600px;
        }

        .control-panel {
            background: #f8f9fa;
            padding: 30px;
            border-right: 2px solid #e9ecef;
        }

        .control-group {
            background: white;
            padding: 20px;
            border-radius: 10px;
            margin-bottom: 20px;
            box-shadow: 0 2px 8px rgba(0,0,0,0.1);
        }

        .control-group h3 {
            color: #667eea;
            margin-bottom: 15px;
            font-size: 1.1em;
        }

        .control-group label {
            display: block;
            margin-bottom: 8px;
            color: #495057;
            font-weight: 500;
        }

        .control-group select {
            width: 100%;
            padding: 12px;
            border: 2px solid #e9ecef;
            border-radius: 8px;
            font-size: 1em;
            transition: all 0.3s;
        }

        .control-group select:focus {
            outline: none;
            border-color: #667eea;
            box-shadow: 0 0 0 3px rgba(102, 126, 234, 0.1);
        }

        .btn {
            width: 100%;
            padding: 15px;
            border: none;
            border-radius: 8px;
            font-size: 1em;
            font-weight: 600;
            cursor: pointer;
            transition: all 0.3s;
            text-transform: uppercase;
            letter-spacing: 0.5px;
        }

        .btn-primary {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            margin-bottom: 10px;
        }

        .btn-primary:hover:not(:disabled) {
            transform: translateY(-2px);
            box-shadow: 0 5px 15px rgba(102, 126, 234, 0.4);
        }

        .btn-primary:disabled {
            background: #ccc;
            cursor: not-allowed;
        }

        .btn-success {
            background: #28a745;
            color: white;
        }

        .btn-success:hover:not(:disabled) {
            background: #218838;
            transform: translateY(-2px);
        }

        .btn-success:disabled {
            background: #ccc;
            cursor: not-allowed;
        }

        .visualization-area {
            padding: 30px;
        }

        .status-card {
            background: #f8f9fa;
            padding: 20px;
            border-radius: 10px;
            margin-bottom: 20px;
            border-left: 4px solid #667eea;
        }

        .status-card.success {
            border-left-color: #28a745;
            background: #d4edda;
        }

        .status-card.error {
            border-left-color: #dc3545;
            background: #f8d7da;
        }

        .mesh-info {
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(150px, 1fr));
            gap: 15px;
            margin-top: 15px;
        }

        .info-item {
            background: white;
            padding: 15px;
            border-radius: 8px;
            text-align: center;
        }

        .info-item .label {
            color: #6c757d;
            font-size: 0.85em;
        }

        .info-item .value {
            color: #667eea;
            font-size: 1.5em;
            font-weight: 700;
        }

        .canvas-container {
            background: #f8f9fa;
            border-radius: 10px;
            padding: 40px;
            text-align: center;
            min-height: 400px;
            display: flex;
            align-items: center;
            justify-content: center;
        }

        .placeholder svg {
            width: 150px;
            height: 150px;
            margin-bottom: 20px;
            opacity: 0.5;
        }

        .loading {
            display: none;
        }

        .loading.active {
            display: block;
        }

        .spinner {
            border: 4px solid #f3f3f3;
            border-top: 4px solid #667eea;
            border-radius: 50%;
            width: 50px;
            height: 50px;
            animation: spin 1s linear infinite;
            margin: 0 auto 20px;
        }

        @keyframes spin {
            0% { transform: rotate(0deg); }
            100% { transform: rotate(360deg); }
        }

        .footer {
            background: #f8f9fa;
            padding: 20px;
            text-align: center;
            color: #6c757d;
        }

        @media (max-width: 968px) {
            .main-content {
                grid-template-columns: 1fr;
            }
        }
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>🫀 3D Heart Reconstruction System</h1>
            <p>Medical Imaging Visualization Platform</p>
        </div>

        <div class="main-content">
            <div class="control-panel">
                <div class="control-group">
                    <h3>🔧 Configuration</h3>
                    <label for="patientSelect">Patient ID</label>
                    <select id="patientSelect">
                        <option value="">Select Patient</option>
                    </select>

                    <label for="modalitySelect" style="margin-top: 15px;">Imaging Modality</label>
                    <select id="modalitySelect">
                        <option value="ct">CT Scan</option>
                        <option value="mr">MRI Scan</option>
                    </select>
                </div>

                <button class="btn btn-primary" id="generateBtn" disabled>
                    Generate 3D Model
                </button>

                <button class="btn btn-success" id="downloadBtn" disabled>
                    📥 Download .OBJ File
                </button>
            </div>

            <div class="visualization-area">
                <div id="statusArea"></div>

                <div class="loading" id="loadingIndicator">
                    <div class="spinner"></div>
                    <p>Generating 3D heart model...</p>
                    <p style="color: #6c757d; margin-top: 10px;">This may take 10-30 seconds</p>
                </div>

                <div class="canvas-container" id="canvasContainer">
                    <div class="placeholder">
                        <svg viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="2">
                            <path d="M20.84 4.61a5.5 5.5 0 0 0-7.78 0L12 5.67l-1.06-1.06a5.5 5.5 0 0 0-7.78 7.78l1.06 1.06L12 21.23l7.78-7.78 1.06-1.06a5.5 5.5 0 0 0 0-7.78z"/>
                        </svg>
                        <h3>Select a patient to begin</h3>
                        <p>Choose patient ID and modality, then click "Generate 3D Model"</p>
                    </div>
                </div>
            </div>
        </div>

        <div class="footer">
            <p>Final Year Engineering Major Project</p>
        </div>
    </div>

    <script>
        let currentMeshData = null;

        const patientSelect = document.getElementById('patientSelect');
        for (let i = 1001; i <= 1010; i++) {
            const option = document.createElement('option');
            option.value = i;
            option.textContent = `Patient ${i}`;
            patientSelect.appendChild(option);
        }

        patientSelect.addEventListener('change', function() {
            document.getElementById('generateBtn').disabled = !this.value;
        });

        document.getElementById('generateBtn').addEventListener('click', async function() {
            const patientId = document.getElementById('patientSelect').value;
            const modality = document.getElementById('modalitySelect').value;

            if (!patientId) {
                alert('Please select a patient ID');
                return;
            }

            document.getElementById('loadingIndicator').classList.add('active');
            document.getElementById('statusArea').innerHTML = '';
            document.getElementById('downloadBtn').disabled = true;
            this.disabled = true;

            try {
                const response = await fetch('/generate', {
                    method: 'POST',
                    headers: {'Content-Type': 'application/json'},
                    body: JSON.stringify({
                        patient_id: parseInt(patientId),
                        modality: modality
                    })
                });

                const data = await response.json();

                document.getElementById('loadingIndicator').classList.remove('active');
                this.disabled = false;

                if (data.success) {
                    currentMeshData = data;
                    displaySuccess(data);
                    document.getElementById('downloadBtn').disabled = false;
                } else {
                    displayError(data.error || 'Failed to generate mesh');
                }
            } catch (error) {
                document.getElementById('loadingIndicator').classList.remove('active');
                this.disabled = false;
                displayError('Network error: ' + error.message);
            }
        });

        document.getElementById('downloadBtn').addEventListener('click', function() {
            if (currentMeshData) {
                window.open(`/download/${currentMeshData.filename}`, '_blank');
            }
        });

        function displaySuccess(data) {
            document.getElementById('statusArea').innerHTML = `
                <div class="status-card success">
                    <h4>✅ 3D Model Generated Successfully</h4>
                    <div class="mesh-info">
                        <div class="info-item">
                            <div class="label">Patient ID</div>
                            <div class="value">${data.patient_id}</div>
                        </div>
                        <div class="info-item">
                            <div class="label">Modality</div>
                            <div class="value">${data.modality.toUpperCase()}</div>
                        </div>
                        <div class="info-item">
                            <div class="label">Vertices</div>
                            <div class="value">${data.vertices_count.toLocaleString()}</div>
                        </div>
                        <div class="info-item">
                            <div class="label">Faces</div>
                            <div class="value">${data.faces_count.toLocaleString()}</div>
                        </div>
                    </div>
                </div>
            `;

            document.getElementById('canvasContainer').innerHTML = `
                <div style="text-align: center;">
                    <svg viewBox="0 0 24 24" fill="none" stroke="#28a745" style="width: 100px; height: 100px; margin-bottom: 20px;" stroke-width="2">
                        <path stroke-linecap="round" stroke-linejoin="round" d="M9 12l2 2 4-4m6 2a9 9 0 11-18 0 9 9 0 0118 0z"/>
                    </svg>
                    <h3 style="color: #28a745;">3D Model Ready!</h3>
                    <p style="color: #495057; margin: 10px 0;">
                        <strong>File:</strong> ${data.filename}
                    </p>
                    <p style="background: #fff3cd; padding: 15px; border-radius: 8px; color: #856404; margin-top: 20px;">
                        💡 Click "Download .OBJ File" above<br>
                        Open in Blender or MeshLab for visualization
                    </p>
                </div>
            `;
        }

        function displayError(message) {
            document.getElementById('statusArea').innerHTML = `
                <div class="status-card error">
                    <h4>❌ Error</h4>
                    <p style="color: #721c24;">${message}</p>
                </div>
            `;
        }
    </script>
</body>
</html>
    '''

@app.route('/api/patients', methods=['GET'])
def get_patients():
    return jsonify({'patients': AVAILABLE_PATIENTS})

@app.route('/generate', methods=['POST'])
def generate_mesh():
    data = request.json
    patient_id = data.get('patient_id')
    modality = data.get('modality', 'ct')

    if not patient_id:
        return jsonify({'success': False, 'error': 'Patient ID is required'}), 400

    result, error = generate_ground_truth_mesh(patient_id, modality)

    if error:
        return jsonify({'success': False, 'error': error}), 500

    return jsonify(result)

@app.route('/download/<filename>', methods=['GET'])
def download_file(filename):
    filepath = os.path.join(OUTPUT_DIR, filename)

    if not os.path.exists(filepath):
        return jsonify({'error': 'File not found'}), 404

    return send_file(filepath, as_attachment=True, download_name=filename)

# ==============================================================================
# START SERVER WITH LOCALTUNNEL (No signup required!)
# ==============================================================================
def run_server():
    app.run(host='0.0.0.0', port=PORT, threaded=True, debug=False)

print("🔧 Installing localtunnel...")
subprocess.run(['npm', 'install', '-g', 'localtunnel'], capture_output=True)

print("🔧 Starting Flask server...")
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

time.sleep(3)

print("🌐 Creating public tunnel with localtunnel...")
print("⏳ Please wait 5-10 seconds...")

# Start localtunnel in background
import subprocess
import re

proc = subprocess.Popen(
    ['lt', '--port', str(PORT)],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

# Wait for URL
public_url = None
for i in range(20):
    line = proc.stdout.readline()
    if line:
        match = re.search(r'https://[^\s]+', line)
        if match:
            public_url = match.group(0)
            break
    time.sleep(0.5)

if not public_url:
    public_url = "Check terminal output for URL"

# ==============================================================================
# DISPLAY ACCESS LINK
# ==============================================================================
print("\n" + "="*80)
print("🚀 3D HEART RECONSTRUCTION SYSTEM IS RUNNING!")
print("="*80)

display(HTML(f'''
<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            padding: 30px;
            border-radius: 15px;
            margin: 20px 0;
            box-shadow: 0 10px 30px rgba(0,0,0,0.3);">
    <h2 style="color: white; margin-bottom: 20px; text-align: center;">
        🫀 3D Heart Reconstruction System
    </h2>
    <div style="background: white; padding: 25px; border-radius: 10px; text-align: center;">
        <h3 style="color: #495057; margin-bottom: 20px;">🌐 Your Application is Ready!</h3>
        <a href="{public_url}" target="_blank"
           style="display: inline-block;
                  background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                  color: white;
                  padding: 20px 50px;
                  text-decoration: none;
                  border-radius: 30px;
                  font-weight: bold;
                  font-size: 1.3em;
                  box-shadow: 0 5px 15px rgba(102, 126, 234, 0.4);">
            🚀 OPEN APPLICATION
        </a>
        <p style="color: #6c757d; margin-top: 20px;">
            <strong>URL:</strong> <code style="background: #f8f9fa; padding: 5px 10px; border-radius: 5px;">{public_url}</code>
        </p>
    </div>
    <div style="margin-top: 20px; padding: 15px; background: rgba(255,255,255,0.1); border-radius: 8px;">
        <p style="color: white; margin: 5px 0;">📁 Output: {OUTPUT_DIR}</p>
        <p style="color: white; margin: 5px 0;">📊 Patients: {AVAILABLE_PATIENTS[0]}-{AVAILABLE_PATIENTS[-1]}</p>
    </div>
</div>

<div style="background: #fff3cd; padding: 20px; border-radius: 10px; margin: 20px 0;">
    <h3 style="color: #856404; margin-bottom: 10px;">📋 How to Use:</h3>
    <ol style="color: #856404; line-height: 1.8; margin-left: 20px;">
        <li>Click "OPEN APPLICATION" button</li>
        <li>Select Patient ID (1001-1010)</li>
        <li>Choose CT or MRI</li>
        <li>Click "Generate 3D Model"</li>
        <li>Download .OBJ file</li>
        <li>Open in Blender/MeshLab</li>
    </ol>
</div>
'''))

print(f"\n✅ Server is ready!")
print(f"🌐 Public URL: {public_url}")
print("="*80 + "\n")

# Keep running
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("\n🛑 Shutting down...")
    proc.kill()

🔧 Installing localtunnel...
🔧 Starting Flask server...
 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


🌐 Creating public tunnel with localtunnel...
⏳ Please wait 5-10 seconds...

🚀 3D HEART RECONSTRUCTION SYSTEM IS RUNNING!



✅ Server is ready!
🌐 Public URL: https://curvy-cats-rest.loca.lt



INFO:werkzeug:127.0.0.1 - - [14/Nov/2025 05:58:07] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Nov/2025 05:58:07] "GET /favicon.ico HTTP/1.1" 404 -


Loading label from: /content/drive/MyDrive/Major_Project/MM-WHS 2017 Dataset/ct_train/ct_train_1010_label.nii.gz
Generating 3D mesh...


INFO:werkzeug:127.0.0.1 - - [14/Nov/2025 05:58:15] "POST /generate HTTP/1.1" 200 -


Mesh saved to: /content/drive/MyDrive/Major_Project/outputs/heart_gt_patient_1010_ct.obj


INFO:werkzeug:127.0.0.1 - - [14/Nov/2025 05:58:20] "GET /download/heart_gt_patient_1010_ct.obj HTTP/1.1" 200 -



🛑 Shutting down...
